In [Fistful of Dice - Best Implementation](https://boardgamegeek.com/thread/2277177/fistful-dice-best-implementation/page/4), Jason Cawley wrote:

> But throw 14 dice vs 12 dice in a Napoleonic Wars battle, and outcomes are possible that are 1/3^26, or about one in 10 trillion to 1 against. “But that won’t happen”, you’ll say. Sure, and nobody will win the lottery tomorrow. Oh right. Napoleon, Lannes, and 7 French armies evaporate; Charles with a 1-4 and 8 Austrians is untouched. 

...
     
> Ken - it’s an actual game, not a hypothetical. All 6s are kills and all 5s are disrupts; if one side’s total of both exceeds the other by 3 or more it is a rout and all the 5s “promote” to 6s aka unit kills. Dice are given for SPs, leaders, and a national bonus (2 French, 1 other major powers, 0 minor powers), so number of dice thrown strictly exceeds number of forces present.

> We all see exactly what that is *meant* to do - give an expectation of 15-20% losses in equal fights, but with 30-40% losses to the losers of outlier routs, that are to become more common as the odds rise. We understand the variance around that is expected to be limited 1-2 SD stuff.

> But the 1 in 10 trillion outcomes being there moves us off that original design intention. If you do the math, the 100% wipeout of the superior French force in the example is about 0.2% or 1/500 to happen. There were a couple hundred major battles the in whole course of the Napoleonic wars (Napoleon was in 50 himself); the event described would be expected never to once in the course of them. 

So, the question is, what is the probability of a Total Army Kill (TAK) of a 7 unit army rolling 14 dice vs. an army rolling 12 dice?

In [1]:
import scipy
from scipy.stats import binom

The probability of rolling *i* successes on *n* dice, where the probability of a success on one dice is *p* is given by the binomial discrete probability mass function.

    binom.pmf(i, n, p)

In [2]:
binom.pmf(1, 1, 1/6) # probablity of rolling a 6 on 1D6.

0.16666666666666669

My plan here is to divide the computation into a (large) number of scenarios, compute the probability of each scenario, then sum the probabilities up to get the overall probability.

The probability of a TAK is 

- the probability of a natual TAK times the probability of not having a rout plus

- the probability of a rout TAK times the probability of a rout.

In the first case, the scenarios have the Austrian player rolling enough kill results to eliminate the French army (7 or more) and also rolling  disrupt results (12 minus the number of kill results or less) so that the total number of French hit results (enemy kills plus disrupts) does not cause a rout (French kills plus disrupts minus two, or more).

In [3]:
naturalKillScenarios = [
    (kills, disrupts, enemyhits)
    for kills in [7, 8, 9, 10, 11, 12]
    for disrupts in range(0, 12 - kills + 1)
    for enemyhits in range(kills + disrupts - 2, 14 + 1)
]
naturalKillScenarios

[(7, 0, 5),
 (7, 0, 6),
 (7, 0, 7),
 (7, 0, 8),
 (7, 0, 9),
 (7, 0, 10),
 (7, 0, 11),
 (7, 0, 12),
 (7, 0, 13),
 (7, 0, 14),
 (7, 1, 6),
 (7, 1, 7),
 (7, 1, 8),
 (7, 1, 9),
 (7, 1, 10),
 (7, 1, 11),
 (7, 1, 12),
 (7, 1, 13),
 (7, 1, 14),
 (7, 2, 7),
 (7, 2, 8),
 (7, 2, 9),
 (7, 2, 10),
 (7, 2, 11),
 (7, 2, 12),
 (7, 2, 13),
 (7, 2, 14),
 (7, 3, 8),
 (7, 3, 9),
 (7, 3, 10),
 (7, 3, 11),
 (7, 3, 12),
 (7, 3, 13),
 (7, 3, 14),
 (7, 4, 9),
 (7, 4, 10),
 (7, 4, 11),
 (7, 4, 12),
 (7, 4, 13),
 (7, 4, 14),
 (7, 5, 10),
 (7, 5, 11),
 (7, 5, 12),
 (7, 5, 13),
 (7, 5, 14),
 (8, 0, 6),
 (8, 0, 7),
 (8, 0, 8),
 (8, 0, 9),
 (8, 0, 10),
 (8, 0, 11),
 (8, 0, 12),
 (8, 0, 13),
 (8, 0, 14),
 (8, 1, 7),
 (8, 1, 8),
 (8, 1, 9),
 (8, 1, 10),
 (8, 1, 11),
 (8, 1, 12),
 (8, 1, 13),
 (8, 1, 14),
 (8, 2, 8),
 (8, 2, 9),
 (8, 2, 10),
 (8, 2, 11),
 (8, 2, 12),
 (8, 2, 13),
 (8, 2, 14),
 (8, 3, 9),
 (8, 3, 10),
 (8, 3, 11),
 (8, 3, 12),
 (8, 3, 13),
 (8, 3, 14),
 (8, 4, 10),
 (8, 4, 11),
 (8, 4, 12),
 (8, 4, 13)

One such scenario has the Austrian player rolling 7 sixes and no fives, while the French player rolls 5 fives or sixes. The next scenario is the same, except the French player rolls 6 fives or sixes.

The probability of each scenario is

- the probability of the Austrian player rolling 
    - the given number of kills on 12 dice, times
    - the given number of disruptions on 12 - the number of kills dice,
- times the probability of the French player rolling
    - the given number of hits on 14 dice.
    
The probability of a natural kill is the sum of the probabilies of the scenarios.

In [4]:
pNaturalKill = sum([
    (
        binom.pmf(kills, 12, 1/6) *
        binom.pmf(disrupts, 12 - kills, 1/6) *
        binom.pmf(enemyhits, 14, 2/6)
    )
    for (kills, disrupts, enemyhits) in naturalKillScenarios
])
pNaturalKill

0.0004427257464841527

Thus, the probability of a natural kill is roughly 1 in 2,259.

Rout scenarios, on the other hand, have the Austrian player rolling enough hits (kills and disrupts) to eliminate the French Army if the French rout, while the French do not roll enough hits to prevent them from routing. In other words, the Austrians roll 7 or more fives or sixes, while the French roll the number of Austrian hits minus 2 or fewer fives or sixes.

In [5]:
routScenarios = [
    (hits, enemyhits)
    for hits in range(7, 12 + 1)
    for enemyhits in range(0, hits - 3 + 1)
]
routScenarios

[(7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (9, 0),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (10, 0),
 (10, 1),
 (10, 2),
 (10, 3),
 (10, 4),
 (10, 5),
 (10, 6),
 (10, 7),
 (11, 0),
 (11, 1),
 (11, 2),
 (11, 3),
 (11, 4),
 (11, 5),
 (11, 6),
 (11, 7),
 (11, 8),
 (12, 0),
 (12, 1),
 (12, 2),
 (12, 3),
 (12, 4),
 (12, 5),
 (12, 6),
 (12, 7),
 (12, 8),
 (12, 9)]

In [6]:
pRoutKill = sum([
    (
        binom.pmf(hits, 12, 2/6) *
        binom.pmf(enemyhits, 14, 2/6)
    )
    for (hits, enemyhits) in routScenarios
])
pRoutKill

0.03628751424249502

This term is quite large, roughly 3.6% or 1 in 28.

In [7]:
pTAK = pNaturalKill + pRoutKill
pTAK

0.03673023998897917

The overall result is about 3.7%. That seems high.